In [ ]:
import pandas as pd

df = pd.read_csv('dummy.csv')
df

In [ ]:
df = df.sample(frac=1).reset_index(drop=True)
df

In [ ]:
df = df.dropna()

In [ ]:
X = df.drop('Labels', axis=1)

In [ ]:
y = df['Labels']

In [ ]:
y1 = df[df['Labels']==0]

In [ ]:
y2 = df[df['Labels']==1]

In [ ]:
df['Labels'].value_counts()

In [ ]:
import tensorflow as tf

In [ ]:
from tensorflow.keras.layers import Embedding, Dropout
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.text import one_hot
from tensorflow.keras.layers import LSTM, GRU, Bidirectional, SimpleRNN, Conv1D, MaxPooling1D, Flatten
from tensorflow.keras.layers import Dense
import keras
from keras import regularizers 
from keras.regularizers import l2
from keras_self_attention import SeqSelfAttention

In [ ]:
voc_size = 13083

In [ ]:
stop_words = pd.read_csv('stwords.csv')
stop_words = stop_words['Words'].tolist()
stop_words.index('a')

In [ ]:
from fuzzywuzzy import fuzz
from fuzzywuzzy import process
import fuzzy

In [ ]:
import nltk
import re
corpus = []

for i in range(0, len(reviews)):
    print(i)
    comment = re.sub('[^a-zA-z]', ' ', reviews['Reviews'][i])
    comment = comment.lower()
    comment = comment.split()
    print(comment)

    
    
    comment = [word for word in comment if not word in stop_words]
    comment = ' '.join(comment)
    corpus.append(comment)
    print("After cleaning")
    print(comment)

In [ ]:
mw = pd.read_csv('most_frequent_words.csv')
mw = mw.dropna()

In [ ]:
mw = mw["Words"].tolist()
mw = set(mw)
mw = list(mw)
mw


In [ ]:
s = 0
updated_corpus = []
dphone = fuzzy.DMetaphone(4)
for i in corpus:
    cor = i.split()
    for j in mw:
        d_1 = dphone(j)
        for k in range(len(cor)):

            d_2 = dphone(cor[k])
            
            if j != cor[k] and fuzz.token_sort_ratio(d_1[0], d_2[0])> 74 and fuzz.token_sort_ratio(j,cor[k])>74 :
                print( j, "--", cor[k])
                

                s =s +1
                cor[k] = j
                
#                 if s>200 and s<251: 
     
                                  
    g = " ".join(cor)
    updated_corpus.append(g)  
        
print(s)

In [ ]:
corpus = updated_corpus
corpus

In [ ]:
mx = max(corpus, key = len)
mx = mx.split()
len(mx)

In [ ]:
onehot_repr = [one_hot(words,voc_size) for words in corpus]
len(corpus)

In [ ]:
onehot_repr = [one_hot(words,voc_size) for words in corpus]
len(corpus)

In [ ]:
sent_length = 131
embedded_docs = pad_sequences(onehot_repr, padding = 'pre', maxlen = sent_length)
embedded_docs

In [ ]:
embedding_vector_features = 60
model = Sequential()
model.add(Embedding(voc_size, embedding_vector_features, input_length= sent_length))
model.add(Conv1D(filters=64, kernel_size=3, padding='same', activation='relu'))
model.add(MaxPooling1D(pool_size=2))


model.add(Conv1D(filters=126, kernel_size=24, padding='same', activation='relu'))
model.add(MaxPooling1D(pool_size=2))
model.add(Dropout(0.2))


model.add((GRU(20)))

model.add(keras.layers.Flatten())

#without svc
model.add(Dense(1, activation ='sigmoid'))
model.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])

print(model.summary())

In [ ]:
import numpy as np
X_final = np.array(embedded_docs)
y_final = np.array(y)


In [ ]:

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_final, y_final, test_size = 0.2, random_state = 42)

In [ ]:
a = model.fit(X_train, y_train, epochs = 10,validation_data = (X_test, y_test), batch_size = 100)

In [ ]:
y_pred = model.predict_classes(X_test)

In [ ]:
y_pred.flatten()